# Strava, Rouvy and Machine Learning

### How to predict 'moving time' on a route by Scikit-Learn
<center><img src="strava.png" alt="My Account from Strava web page" /><img src="rouvy.png" alt="My Account from Strava web page" /></center>

I've had a lot of fun riding my bike over the last few years. Unfortunately the Covid pandemic has greatly limited the opportunities for outdoor outings. Plus the winter is harsh in my area. So I subscribed to a nice app called (rouvy.com) and bought an indoor trainer to pedal at home. 

It was a fantastic experience that continues to this day. Early every day in the morning I can train by pedaling anywhere in the world, tackling the steepest and most legendary climbs. 

I connected Rouvy to my (free) Strava account so that every route I ride under Rouvy is automatically saved to Strava. After 3 years the end result is that I have more than 500 routes (indoor and outdoor) saved in my Strava account. But how does Machine Learning come into play?

When I want to choose the next route to take, I often have only a rough idea of the "commute time" it will take. It would be helpful to have some Moving time prediction to better schedule my time. So I decided to use the data available in Strava to train some Machine Learning models and predict the "commute time" (also 'Moving time') given some parameters (distance, elevation gain, max grade, average grade...) of the route. This data are available 'a priori' in the Rouvy profile of the route.
The notebook, the data and all the pictures are available under my github https://github.com/fabioantonini/strava-moving-time-regressor.

## Outline
Here the topics we are going to talk.

- ### Retrieving data from Strava
- ### Data Exlporation
- ### Data Cleaning
- ### Selecting Features and Labels
- ### Outliers
- ### Save the cleaned data
- ### Data Visualization
- ### Looking for correlation
- ### Avoiding sampling bias
- ### Splitting Training and Test sets
- ### Linear Regression Model
- ### Challenge Gunsan-Saemangeum 2002 prediction
- ### Decision Tree Model
- ### Nusfjord to Haukland Beach | Norway prediction
- ### Conclusions

## Retrieving data from Strava

The routes data can be exported by the Strava website from the 'My Account' page.

<center><img src="myaccount.png" alt="My Account from Strava web page" /></center>

Search for 'Download or Delete Your Account'. Click on the 'Get Started'.

<img src="export.png" alt="My Account from Strava web page" />

Click on the 'Request Your Archive' button. As explained, an email will be sent to you with the link to download the zip file containing the data of your Strava activities. Prepare to wait for a while. Strava takes its time to arrange the archive. So you might receive the email after some hours.
Anyway in the end you will receive the email and download the zip file (export_31174850.zip for instance).
Let's take a look at its content.

<img src="zipfile.png" alt="My Account from Strava web page" />

For our purposes only the 'activities.csv' file is required. From the size we can realize that it contains a lot of data. My own 'activities.csv' file has been added to the repo and it will be processed next. Let's import it.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
from sklearn.linear_model import LinearRegression # Regression Model
from sklearn.model_selection import train_test_split # to split train and test sets
plt.style.use("bmh")
%config InlineBackend.figure_formats=["png"]

In [ ]:
activities = pd.read_csv("activities.csv")
print("dataset type is:", type(activities), "length:", len(activities), "shape:", activities.shape)

## Data exploration

The dataset is made of 855 activities (rows), but unfortunately not all of them are rides by bike.

The single route (row) includes 86 columns. Not all the columns contain usable data (many NaN or 'null' are present) because I don't have a full Strava subscription, but only a free account. 

Let's take a look more in depth to undestand which activities are really useful to our purpose.

In [ ]:
print("columns: ", len(list(activities.columns)))

In [ ]:
activities.head()

In [ ]:
activities.info()

We need to extract only the columns really useful to train a model.
The data appear to be a bit sparsed. Some columns are not valorized at all (because of my free account). Others columns have many 'null' values. We need to identify only the features statistically helpful that are available for each route before riding the route itself.

Let's clean the data.

## Data cleaning

In the next section data will be cleaned and filtered to get only routes done by bike (Outdoor and Indoor).

### Getting only activities done by bike

We defintely need to get only the activities done by bike. They are labeled as 'Ride' and 'Virtual Ride' in the Strava exported dataset. So we will drop the activities tagged as 'Walk' and 'Run'.

In [ ]:
activities=activities.loc[activities['Activity Type'].isin(['Ride', 'Virtual Ride'])]

In [ ]:
activities.describe()

You can notice that the number of rows decreased a bit. Let's go further.

In [ ]:
# as an alternative you can remove the 'Walk' and 'Run' activities
# activities = activities.drop(activities[activities['Activity Type'] == 'Walk'].index)
# activities = activities.drop(activities[activities['Activity Type'] == 'Run'].index)

### Removing short routes

When you ride under Rouvy you can optionally have a 'Warm up' and 'Cool down' before and after respectively the selected route. Also these short routes have been recorded under Strava. They are not useful for our purposes. So let's remove all the routes whose 'Moving Time' is less than 3 minutes (180 secs).

In [ ]:
activities = activities.drop(activities[activities['Moving Time'] < 180].index)

In [ ]:
activities.describe()

The count number has decreased again.

### Handling fake data

Inspecting the original dataframe we can realize that there are some bad data. For instance it's hard to believe that the 'Max Grade' is 50%. If the min 'Distance' is 0 Km, the route is a 'fake' or the data are corrupted. So these routes can be removed.
Let's clean this data by setting a threshold of 25% for the 'Max Grade', and 3 Km's for 'Distance' respectively.

In [ ]:
activities = activities.drop(activities[activities['Max Grade'] > 25].index)

In [ ]:
activities = activities.drop(activities[activities['Distance'] < 3].index)

In [ ]:
activities.describe()

Now we have 533 bike routes (Indoor and Outdoor). Let's take a look.

In [ ]:
activities.head()

Now the data looks better. It's time to get the features really helpful to train our models.

## Handling 'Null' values

We can check the 'null' values.

In [ ]:
null_rows_idx = activities.isnull().any(axis=1)
activities.loc[null_rows_idx].head()

We have a couple of policies to handle the 'Null' values.

We can use 'imputation' to set the NaN to the median value of that feature. A more rude approach is to remove the rows with at least one NaN or null value, but in this  case we will lost some data.

Let's try to use imputation in order to save the three rows with NaN values.

#### Imputation

In [ ]:
median = activities["Elevation Gain"].median()
activities["Elevation Gain"].fillna(median, inplace=True)  # option 3

median = activities["Max Grade"].median()
activities["Max Grade"].fillna(median, inplace=True)  # option 3

activities.loc[null_rows_idx].head()

#### Droping NaN

You can remove the rows with at least one 'null' value.

In [ ]:
#if activities.isnull().values.any():
#    activities=activities.dropna()

In [ ]:
activities.describe()

We have 533 records (rows) that can be used to train our models.

## Selecting features and labels

After a fast analysis of the available features, only the following features will be used to train the model:
- Distance
- Elevation gain
- Max Grade
- Average Grade

The target label is the 'Moving Time'.

These informations can be retrieved by Rouvy, for every available route.
Please note that in the picture the Elevation Gain is mapped to the 'Ascended' item.

In the next predictions examples we will get this data from the Route info under Rouvy and use them to make prediction of the 'Moving Time' for some routes not included in the original dataset.

<center><img src="rouvy-route.png" alt="Rouvy data for a route" /></center>

In [ ]:
activities = activities[["Distance", "Elevation Gain", "Max Grade", "Average Grade", "Moving Time"]]
activities.describe()

## Outliers

Now let's drop some outliers:

In [ ]:
from sklearn.ensemble import IsolationForest

isolation_forest = IsolationForest(random_state=42)
outlier_pred = isolation_forest.fit_predict(activities)

In [ ]:
#outlier_pred

In [ ]:
#activities = activities.iloc[outlier_pred == 1]
#activities.describe()

## Save the cleaned data

Now the dataset has been cleaned and filtered. We will develop some models using this data.
The dataframe can be stored to the filesystem.

In [ ]:
activities.to_csv('cleaned_activities.csv')

In [ ]:
activities.head()

Let's take a look at the dataframe after the last processing.

## Visualization

We can obtain a first impression of the dependency between variables by examining a multidimensional scatterplot.

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(activities, diagonal="kde", figsize=(12,10));

As expected we can see a linear relationship between the Moving Time and the Distance.

In [ ]:
activities.plot(kind="scatter", x='Distance', y='Moving Time', grid=True)

there is an approximately linear relationship between Elevation Gain and the Distance: more Kms more the overall gain in altitude

In [ ]:
activities.plot(kind="scatter", x='Distance', y='Elevation Gain', grid=True)

We can also generate a 3D plot of the observations, which can sometimes help to interpret the data more easily. Here we plot 'Moving Time' as a function of 'Distance' and 'Elevation Gain'.

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(projection="3d")
ax.scatter(activities["Distance"], activities["Elevation Gain"], activities["Moving Time"])
ax.set_xlabel("Distance")
ax.set_ylabel("Elevation Gain")
ax.set_zlabel("Moving Time")
ax.set_facecolor("white")

In [ ]:
%matplotlib inline
activities.hist(bins=50, figsize=(20,15))
plt.show()

## Looking for correlation

You can easily compute the standard correlation coefficient (also called Pearson's r) between every pair of attributes using the 'corr()' method.

In [ ]:
corr_matrix= activities.corr()

In [ ]:
corr_matrix["Moving Time"].sort_values(ascending=False)

The Moving Time is strongly correlated to the 'Distance' and also to the 'Elevation Gain'. This is asbolutely expected.

## Avoiding sampling bias

Before splitting the dataset into a Training and a Test set we need to face the problem of 'Sampling bias'. Usually we can use a random samoling approach. This is generally fine if your dataset is large enough (especially relative to the number of attributes), but if it is not, you run the risk of introducing a significant sampling bias. In our case the dataset is quite small. The risk to face sampling bias is high. We need a workaround.

From the previous histograms we can notice that most 'Distance' values are clustered around 10 to 15 Km's, but some 'Distance's go far beyond 70. It is important to have a sufficient number of instances in your dataset for each stratum, or else the estimate of the stratum’s importance may be biased. This means that you should not have too many strata, and each stratum should be large enough. The following code uses the 
pd.cut() function to create an income category attribute with 5 categories (labeled
from 1 to 6): category 1 ranges from 0 to 20 (i.e., less than 20 Km's), category 2 from
20 to 40 Km's, and so on:

In [ ]:
activities["Distance_cat"] = pd.cut(activities["Distance"],
 bins=[0, 20, 40, 60, np.inf],
 labels=[1, 2, 3, 4])

In [ ]:
activities["Distance_cat"].hist()

Now you are ready to do stratified sampling based on the income category. For this
you can use Scikit-Learn’s StratifiedShuffleSplit class:

In [ ]:
activities = activities.reset_index()

In [ ]:
activities.pop('index')

In [ ]:
activities.head()

In [ ]:
activities.index

## Splitting Training and Test sets

Now it's time to get our Training and Test sets.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
splitter = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
strat_splits = []
for train_index, test_index in splitter.split(activities, activities["Distance_cat"]):
    strat_train_set_n = activities.iloc[train_index]
    strat_test_set_n = activities.iloc[test_index]
    strat_splits.append([strat_train_set_n, strat_test_set_n])

In [ ]:
strat_train_set, strat_test_set = strat_splits[0]

In [ ]:
strat_train_set.describe()

In [ ]:
strat_test_set.describe()

It's much shorter to get a single stratified split:

In [ ]:
strat_train_set, strat_test_set = train_test_split(
    activities, test_size=0.2, stratify=activities["Distance_cat"], random_state=42)

Let's extract the labels for the Training and Test sets

In [ ]:
strat_train_set_labels=strat_train_set.pop("Moving Time")

In [ ]:
strat_test_set_labels=strat_test_set.pop("Moving Time")

In [ ]:
strat_train_set.info()

In [ ]:
strat_test_set.info()

In [ ]:
strat_test_set["Distance_cat"].value_counts() / len(strat_test_set)

In [ ]:
# extra code – computes the data for Figure 2–10

def distance_cat_proportions(data):
    return data["Distance_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(activities, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({
    "Overall %": distance_cat_proportions(activities),
    "Stratified %": distance_cat_proportions(strat_test_set),
    "Random %": distance_cat_proportions(test_set),
}).sort_index()
compare_props.index.name = "Distance Category"
compare_props["Strat. Error %"] = (compare_props["Stratified %"] /
                                   compare_props["Overall %"] - 1)
compare_props["Rand. Error %"] = (compare_props["Random %"] /
                                  compare_props["Overall %"] - 1)
(compare_props * 100).round(2)

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("Distance_cat", axis=1, inplace=True)

## Linear Regression Model

We will created a fitted linear model using the formula API of the scikit-learn library.

In [ ]:
linear_model = LinearRegression()
linear_model.fit(strat_train_set, strat_train_set_labels) 

### View Parameters 
The $\mathbf{w}$ and $\mathbf{b}$ parameters are referred to as 'coefficients' and 'intercept' in scikit-learn. In other term the model function can be written as $f_{w,b}(\vec{x})$

In [ ]:
b = linear_model.intercept_
w = linear_model.coef_
print(f"w = {w:}, b = {b:0.2f}")

Let's give it a try by using some routes from the Test set.

In [ ]:
some_data=strat_test_set.iloc[0:5,:]

In [ ]:
some_labels=strat_test_set_labels.iloc[0:5]
some_labels.head()

In [ ]:
print(type(some_data))
some_labels_predicted = linear_model.predict(some_data) # it return a numpy ndarray
some_labels_predicted

In [ ]:
error = some_labels_predicted - some_labels.to_numpy()

In [ ]:
print("Error (mins):", error/60)

### Mean squared error

Let’s measure the regression model’s RMSE on the whole training set using Scikit-Learn’s mean_squared_error function.

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
lin_mse = mean_squared_error(some_labels, some_labels_predicted)
lin_rmse = np.sqrt(lin_mse)
print("rmse (min)= {}".format(lin_rmse/60))

## Calculate accuracy

You can calculate this accuracy of this model by calling the `score` function on the whole Training set.

In [ ]:
print("Accuracy on training set:", linear_model.score(strat_train_set, strat_train_set_labels))

The model is doing poor on the Training set. It's too simple and it cannot match the training data at the best. Let's try another model.

In [ ]:
print("Accuracy on test set:", linear_model.score(strat_test_set, strat_test_set_labels))

## Challenge Gunsan-Saemangeum 2002 prediction

Let's try to predict the Moving Time of a new route I rode the last weew, the Challenge Gunsan-Saemangeum 2022.

The input data are:
- Distance: 30 Km's
- Elevation Gain: 26
- Max Grade: 3%
- Average Grade: 0%

The real Moving Time is 53 minutes

In [ ]:
real_moving_time=53

<center><img src="test-route.png" alt="Test Route - Challenge Gunsan-Saemangeum 2022" /></center>

In [ ]:
route_data = pd.DataFrame({"Distance": [29.99], "Elevation Gain": [26], "Max Grade": [3], "Average Grade": [0]})

In [ ]:
predicted_moving_time = linear_model.predict(route_data)/60

In [ ]:
print("Predicted Moving Time (mins):{}, error : {}%".format(predicted_moving_time, abs(100*(predicted_moving_time-real_moving_time)/real_moving_time)))

## Decision tree Model

In order to try to improve the Model accuracy let's a different model able to catch nonlinear patterns in the data.
Let’s train a DecisionTreeRegressor. This is a powerful model, capable of finding complex nonlinear relationships in the data.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(strat_train_set, strat_train_set_labels)

In [ ]:
strat_train_set_predictions = tree_reg.predict(strat_train_set)
tree_mse = mean_squared_error(strat_train_set_labels, strat_train_set_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

In [ ]:
print("Accuracy on training set:", tree_reg.score(strat_train_set, strat_train_set_labels))

In [ ]:
print("Accuracy on test set:", linear_model.score(strat_test_set, strat_test_set_labels))

Let's try to predict the 'Moving Time of the previous route.

In [ ]:
predicted_moving_time = tree_reg.predict(route_data)/60

In [ ]:
print("Predicted Moving Time (mins):{}, error : {}%".format(predicted_moving_time, abs(100*(predicted_moving_time-real_moving_time)/real_moving_time)))

Now the error is lower than the error of the Linear Model.

In [ ]:
from sklearn.tree import export_graphviz

In [ ]:
export_graphviz(
 tree_reg,
 out_file="routes_tree.dot",
 feature_names=["Distance", "Elevation Gain", "Max Grade", "Average Grade"],
 class_names="Moving Time",
 rounded=True,
 filled=True
)

In [ ]:
! dot -Tpng routes_tree.dot -o routes_tree.png

<center><img src="routes_tree.png" alt="Decision Tree for the 'Moving Time' regresso" /></center>

The RMSE of the Decision Tree model is '0'. No errors. This is confirmed by the Accuracy calculation on the Training Set which is '1'.

The Accuracy on the Test set is bit lower, but high enough.

The 'Moving Time' of the 'Challenge Gunsan-Saemangeum 2022' Route has been predicted with a lower error %.

## Nusfjord to Haukland Beach | Norway prediction

Let's try to predict the Moving Time of a new route I rode recently, the Nusfjord to Haukland Beach in Norway.

The input data are:

- Distance: 28.62 Km's
- Elevation Gain: 303
- Max Grade: 9%
- Average Grade: 1%

The real Moving Time is 54 minutes

In [ ]:
real_moving_time=54

In [ ]:
nusfjord_route_data = pd.DataFrame({"Distance": [28.62], "Elevation Gain": [303], "Max Grade": [9], "Average Grade": [1]})

In [ ]:
predicted_moving_time = tree_reg.predict(nusfjord_route_data)/60

In [ ]:
print("Predicted Moving Time (mins):{}, error : {}%".format(predicted_moving_time, abs(100*(predicted_moving_time-real_moving_time)/real_moving_time)))

## Conclusions

The purpose of this exercise was to to develop a Machine Learning Model to predict the 'Moving Time' of a Route based. 
- Data includes more of 500 'outdoor' and 'indoor' routes. 
- The 'indoor' routes have been exported by Rouvy to Strava.
- The 'outdoor' routes have been recorded directly by Strava
- The data have been cleaned and prepared for training.
- A couple of Regressors have been trained on the prepared data
- A Linear Regressor prooved to be good, but unable to perfectly fit the training dataset. The accuracy on the training was comparable with the test set one. There was room for some improvement. Maybe the linear model didn't fit at the best.
- The Decision Tree showed a better result to predict the Moving time of an 'unseen' new route. The Decision Tree model has been able to predict the Moving Time of a new route (not seen at all during the training, with an error of 5 minutes (8%) on a route of 53 minutes as 'Moving Time'.